# OvP project


## Imports and data loading

In [1]:
# importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import yake
# import rake
# from rake import RAKE
# from rake_nltk import Rake
# from keybert import KeyBERT
# from langdetect import detect
# import nltk
# import re
# from nltk.corpus import stopwords
# from nltk.tokenize import sent_tokenize
# from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
# 
# import spacy
# import keyword_spacy
# from keyword_spacy import KeywordExtractor

# import openai
from keybert.llm import OpenAI
from keybert import KeyLLM

# nltk.download('stopwords')
# nltk.download('punkt_tab')
# nltk.download('punkt')
import transformers
from transformers import pipeline

In [2]:
f = open("C:\\Users\sue-e\Downloads\openAI API key.txt")
GPT_API_KEY = f.read()

## Osiris excel data

In [3]:
# import data
data = pd.read_excel("raw_data.xlsx")

data.head(2)

,CURSUS,LANGE_NAAM_NL,DOCENT_ROL,OSS_ADF_UTILITY.HTML_TO_TEXT(H.INHOUD)
0,ASB-ACT-EN,ACT - Leef zoals je wilt,Brasjen DOCENT; Bruinsma DOCENT; Buiks DOCENT;...,NaN
1,ASB-ACTLEEF,ACT - Leef zoals je wilt,Brasjen DOCENT; Bruinsma DOCENT; Buiks DOCENT;...,NaN


In [4]:
# format teachers

def format_teachers(row):
    teachers = row['DOCENT_ROL']
    # 'Brasjen DOCENT; Bruinsma DOCENT; Buiks DOCENT; Burgh DOCENT; Galipò DOCENT; Gronden DOCENT; Kaathoven CONTACTPERSOON; Verhagen DOCENT'
    
    formatted_list = []
    
    
    teachers = teachers.split(";")
    roles= ['DOCENT', 'TEACHER', 'EXAMINATOR', 'EXAMINER']
    
    for teach in teachers:
        t = teach.split()
        
        if t[-1] in roles:
            formatted_list.extend(t[:-1])
            
    delimiter = "; " # Define a delimiter
    join_str = delimiter.join(formatted_list)
    return join_str

data['DOCENT_ROL'] = data.apply(format_teachers, axis=1)

data.head(2)

,CURSUS,LANGE_NAAM_NL,DOCENT_ROL,OSS_ADF_UTILITY.HTML_TO_TEXT(H.INHOUD)
0,ASB-ACT-EN,ACT - Leef zoals je wilt,Brasjen; Bruinsma; Buiks; Burgh; Galipò; Grond...,NaN
1,ASB-ACTLEEF,ACT - Leef zoals je wilt,Brasjen; Bruinsma; Buiks; Burgh; Galipò; Grond...,NaN


In [5]:
OG_data = data.copy()
small_data = data.iloc[29995:3000]

## Pipeline creation for specialty extraction

In [6]:
# load model
# extractor = pipeline("text-generation", model="microsoft/Phi-3-mini-4k-instruct")
extractor = pipeline("text-generation", model="microsoft/Phi-3-mini-4k-instruct")


# Define your extraction function
def extract_specialty(row):
    prompt = f"""
    You are an academic data extractor.

    From the following course metadata, identify the teacher's likely area of academic specialization.
    Focus on the academic field or topic expertise (max 7 words).
    All teachers in the same course get the same specialty added.
    If there is no course description or if the course description is nan, extract the teachers but make their specialty blank. 

    Return JSON with keys: name, specialty.


    Course code: {row['CURSUS']}
    Course name: {row['LANGE_NAAM_NL']}
    Teacher and role: {row['DOCENT_ROL']}
    Course description: {row['OSS_ADF_UTILITY.HTML_TO_TEXT(H.INHOUD)']}
    """

    try:
        result = extractor(prompt, max_new_tokens=100)[0]["generated_text"]
        # print(result)
        return result
    except Exception as e:
        print("\nFAIl")
        return str(e)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cpu


In [7]:
# test case
new_data = extract_specialty(data.iloc[2998])
print(new_data)


    You are an academic data extractor.

    From the following course metadata, identify the teacher's likely area of academic specialization.
    Focus on the academic field or topic expertise (max 7 words).
    All teachers in the same course get the same specialty added.
    If there is no course description or if the course description is nan, extract the teachers but make their specialty blank. 

    Return JSON with keys: name, specialty.


    Course code: SOW-BKI323
    Course name: Brain-Computer Interfacing
    Teacher and role: Tangermann; Thielen
    Course description: Content
What the course is about: This course is centered around the data analysis pipelines typically found in brain-computer interfaces: the processing of multi-channel recordings of neuronal signals, the extraction of informative features (“neural markers”) from these signals, and the decoding of brain states by regression or classification models. As informative features are subject-dependent, and as n

In [8]:
# small datset
small_data["structured_info"] = small_data.apply(extract_specialty, axis=1)

small_data.to_excel("created_data/teachers_with_specialties_local.xlsx", index=True)


In [9]:
print("done")

done


## Word document

In [10]:
# from spire.doc import *
# from spire.doc.common import *
from docx import Document

In [11]:
# spire lib
# doc = Document()
# doc.LoadFromFile("Radboud AI - samenvatting - activiteiten en groepen.docx")
# 
# builtinProperties = doc.BuiltinDocumentProperties

# properties = [
#     "Author: " + builtinProperties.Author,
#     "Company: " + builtinProperties.Company,
#     "Title: " + builtinProperties.Title,
#     "Subject: " + builtinProperties.Subject,
#     "Keywords: " + builtinProperties.Keywords,
#     "Category: " + builtinProperties.Category,
#     "Manager: " + builtinProperties.Manager,
#     "Comments: " + builtinProperties.Comments,
#     "Hyperlink Base: " + builtinProperties.HyperLinkBase,
#     "Word Count: " + str(builtinProperties.WordCount),
#     "Page Count: " + str(builtinProperties.PageCount),
# ]
# 
# for i in range(0, len(properties)):
#     print(properties[i])

In [12]:
# # Iterate through the sections
# for i in range(doc.Sections.Count):
#     # Get a specific section
#     section = doc.Sections.get_Item(i)
# 
#     # Get tables from the section
#     tables = section.Tables
# 
#     # Iterate through the tables
#     for j in range(0, tables.Count):
# 
#         # Get a certain table
#         table = tables.get_Item(j)
# 
#         # Declare a variable to store the table data
#         tableData = ""
# 
#         # Iterate through the rows of the table
#         for m in range(0, table.Rows.Count):
# 
#             # Iterate through the cells of the row
#             for n in range(0, table.Rows.get_Item(m).Cells.Count):
# 
#                 # Get a cell
#                 cell = table.Rows.get_Item(m).Cells.get_Item(n)
# 
#                 # Get the text in the cell
#                 cellText = ""
#                 for para in range(cell.Paragraphs.Count):
#                     paragraphText = cell.Paragraphs.get_Item(para).Text
#                     cellText += (paragraphText + "     ")
# 
#                 # Add the text to the string
#                 tableData += cellText
# 
#             # Add a new line
#             tableData += "\n"
#     
#         # Save the table data to a text file
#         try:
#             with open(f"output/WordTable_{i+1}_{j+1}.txt", "w", encoding="utf-8") as f:
#                 f.write(tableData)
#                 f.close()
#         except:
#             print("FAIl")

In [13]:
# text = doc.GetText()
# 
# print(text)

### Python-docx library

In [14]:
doc = Document("Radboud AI - samenvatting - activiteiten en groepen.docx")

for t, table in enumerate(doc.tables):
    print(f"--- Tabel {t+1} ---")
    for row in table.rows:
        cells = [cell.text.strip() for cell in row.cells]
        print(cells)

--- Tabel 1 ---
['Data Science (onderdeel van ICIS)', 'Profiel [max. 250 woorden, excl. namen onderzoekers en ratings/aantallen]\nHet onderzoek richt zich op machinaal leren en kunstmatige intelligentie in het algemeen, met toepassingen in (onder andere) de neurowetenschappen en bioinformatica. De groep, onder leiding van Prof. Djoerd Hiemstra, bestaat uit ongeveer 50 onderzoekers.\nWe ontwikkelen theorie en methoden voor schaalbare machine learning en information retrieval om big data te analyseren en uitdagende problemen in de wetenschap en maatschappij aan te pakken. We zijn betrokken bij verschillende projecten met andere groepen, zowel binnen als buiten de Radboud Universiteit. Onderzoek financiering komt voornamelijk van NWO, TTW en de EU. We maken deel uit van Radboud AI en dragen bij aan Radboud AI for Health.\nProf. dr. Tom Heskes [41]\nProf. dr. Elena Machiori \nProf. dr. Martha Larson [34]\nProf. dr. ir. Arjen de Vries (Spinque) [72]\nProf. dr. ir. Djoerd Hiemstra [24]\nProf

## webscraper tutorials

-  https://medium.com/@joerosborne/intro-to-web-scraping-build-your-first-scraper-in-5-minutes-1c36b5c4b110
- https://medium.com/@joerosborne/web-scraping-with-puppeteer-extra-typescript-aws-lambda-bf4f49d49806
